In [19]:

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import random
import statistics
Xtrain = pd.read_csv('./X_train.csv')
res = pd.read_csv('./y_train.csv')
Xtrain["response"] = res["y"]
Xtest = pd.read_csv('./X_test.csv')
me = res["y"].mean()
sd1 = res["y"].std()
Xtest = (Xtest - Xtrain.mean())/Xtrain.std()
Xtrain = (Xtrain - Xtrain.mean())/Xtrain.std()
tr = []
te = []
for i in range(int(0.7*8000)):
    x=random.randint(0,7999) 
    if x in tr:
        i-=1
        continue
    tr+=[x]
for i in range(int(0.3*8000)):
    x = random.randint(0,7999)
    if x in tr or x in te:
        i-=1
        continue
    te+=[x]   
train = Xtrain.iloc[tr]
test = Xtrain.iloc[te]
# Define the neural network model
class Net(nn.Module):
    def __init__(self, ll=0.1):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(21, int(256*1))
        self.fc2 = nn.Linear(int(256*1), int(128*1))
        self.fc3 = nn.Linear(int(128*1), int(64*1))
        self.fc4 = nn.Linear(int(64*1), 1)
        self.activation = nn.LeakyReLU(ll)

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x


# Initialize the model

my_list= [Xtrain["BMXHT"],
          Xtrain["BMXWT"],
          Xtrain["RIAGENDR"],
          Xtrain["DR1TSUGR"],
          Xtrain["DMDBORN4"],
          Xtrain["SMAQUEX2"],
          Xtrain["BPACSZ"],
          Xtrain["DMDHRGND"],
          Xtrain["DR1TTFAT"],
          Xtrain["BMXARML"],
          Xtrain["DMDHHSIZ"],
          Xtrain["BMXWAIST"],
          Xtrain["BMXSAD1"],
          Xtrain["LBDTCSI"],
          Xtrain["DR1TALCO"],
          Xtrain["BPXPLS"],
          Xtrain["BMXBMI"],
          Xtrain["BMXLEG"],
          Xtrain["BPXPTY"],
          Xtrain["WTMEC2YR"],
          Xtrain["DR1BWATZ"]]
my_listtr= [train["BMXHT"],
          train["BMXWT"],
          train["RIAGENDR"],
          train["DR1TSUGR"],
          train["DMDBORN4"],
          train["SMAQUEX2"],
          train["BPACSZ"],
          train["DMDHRGND"],
          train["DR1TTFAT"],
          train["BMXARML"],
          train["DMDHHSIZ"],
          train["BMXWAIST"],
          train["BMXSAD1"],
          train["LBDTCSI"],
          train["DR1TALCO"],
          train["BPXPLS"],
          train["BMXBMI"],
          train["BMXLEG"],
          train["BPXPTY"],
          train["WTMEC2YR"],
          train["DR1BWATZ"]]
my_listte= [test["BMXHT"],
          test["BMXWT"],
          test["RIAGENDR"],
          test["DR1TSUGR"],
          test["DMDBORN4"],
          test["SMAQUEX2"],
          test["BPACSZ"],
          test["DMDHRGND"],
          test["DR1TTFAT"],
          test["BMXARML"],
          test["DMDHHSIZ"],
          test["BMXWAIST"],
          test["BMXSAD1"],
          test["LBDTCSI"],
          test["DR1TALCO"],
          test["BPXPLS"],
          test["BMXBMI"],
          test["BMXLEG"],
          test["BPXPTY"],
          test["WTMEC2YR"],
          test["DR1BWATZ"]]
# Train the model
n_epoch = 10000


# Get predictions from the random forest

model = Net(0.125)

# Define the loss function and optimizer
criterion = nn.L1Loss()
optimizer = optim.SGD(model.parameters(), lr=0.015)
prev = (0,100)
for epoch in range(n_epoch):
    # Forward pass
   # inputs = torch.Tensor(np.array(my_list)).T # Transpose the input tensor
    inputs = torch.Tensor(np.array(my_listtr)).T # Transpose the input tensor
    #inputs = torch.Tensor(np.array(Xtrain.loc[:, Xtrain.columns != 'response'])).T # Transpose the input tensor
   # labels = torch.Tensor(Xtrain["response"].values).view(-1, 1)
    labels = torch.Tensor(train["response"].values).view(-1, 1)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    # Backward and optimize
    loss.backward()
    optimizer.step()
    
    # Print progress
    if epoch % (n_epoch//30) == 0:
        test_tensor = torch.from_numpy(np.array(my_listte).T).float()
        predictions = model(test_tensor)

        # convert the predictions to a numpy array
        predictions = (predictions.detach().numpy() * sd1) + me
        err = 0
        for i in range(len(predictions)):
            temp = (test.iloc[i]["response"]* sd1) + me
            err+=((predictions[i])-temp)**2
        err/=len(predictions)
        print('Epoch [{}/{}], MSE: {}'.format(epoch+1, n_epoch, str(err)))
        if (err > prev[1]): 
            print ("Overfit at epoch {}, min err = {}, optimal epoch = {}".format(epoch+1,str(prev[1]),str(prev[0])))
            break
        else: prev = (epoch,err)
print("Best epoch is",str(prev[0]), "with MSE",str(prev[1]))
# Save the model
torch.save(model.state_dict(), 'my_model.pt')

Epoch [1/10000], MSE: [17.518026]
Epoch [334/10000], MSE: [11.778165]
Epoch [667/10000], MSE: [10.322196]
Epoch [1000/10000], MSE: [9.553412]
Epoch [1333/10000], MSE: [8.89816]
Epoch [1666/10000], MSE: [8.413203]
Epoch [1999/10000], MSE: [8.050415]
Epoch [2332/10000], MSE: [7.790544]
Epoch [2665/10000], MSE: [7.649035]
Epoch [2998/10000], MSE: [7.5537977]
Epoch [3331/10000], MSE: [7.468132]
Epoch [3664/10000], MSE: [7.418226]
Epoch [3997/10000], MSE: [7.34878]
Epoch [4330/10000], MSE: [7.346429]
Epoch [4663/10000], MSE: [7.2948003]
Epoch [4996/10000], MSE: [7.294324]
Epoch [5329/10000], MSE: [7.2758327]
Epoch [5662/10000], MSE: [7.2890677]
Overfit at epoch 5662, min err = [7.2758327], optimal epoch = 5328
Best epoch is 5328 with MSE [7.2758327]


In [9]:
# read in the CSV file


# convert the test data to a PyTorch tensor
my_list2= [Xtest["BMXHT"],
          Xtest["BMXWT"],
          Xtest["RIAGENDR"],
          Xtest["DR1TSUGR"],
          Xtest["DMDBORN4"],
          Xtest["SMAQUEX2"],
          Xtest["BPACSZ"],
          Xtest["DMDHRGND"],
          Xtest["DR1TTFAT"],
          Xtest["BMXARML"],
          Xtest["DMDHHSIZ"],
          Xtest["BMXWAIST"],
          Xtest["BMXSAD1"],
          Xtest["LBDTCSI"],
          Xtest["DR1TALCO"],
          Xtest["BPXPLS"],
          Xtest["BMXBMI"],
          Xtest["BMXLEG"],
          Xtest["BPXPTY"],
          Xtest["WTMEC2YR"],
          Xtest["DR1BWATZ"]]
Xtest_tensor = torch.from_numpy(np.array(my_list2).T).float()
#Xtest_tensor = torch.from_numpy(np.array(Xtest.loc[:, Xtest.columns != 'response'])).float()

ysimp = pd.read_csv("./y_sample.csv")
seq = ysimp["SEQN"]
# make predictions on the test data using the trained model
predictions = model(Xtest_tensor)

# convert the predictions to a numpy array
predictions = (predictions.detach().numpy() * sd1) + me

# save the predictions to a CSV file

prediction = pd.DataFrame()
prediction['SEQN']=seq 
prediction['y']=predictions
prediction.to_csv('prediction.csv')